In [1]:
import pymysql
import pandas as pd

# Configuración BD (misma que tienes en tu código original)
DB_HOST = "192.168.245.33"
DB_USER = "compensaciones_rrhh"
DB_PASSWORD = "_Cramercomp2025_"
DB_NAME = "rrhh_app"

In [ ]:
def conectar_y_extraer_datos():
    """
    Conecta a la base de datos y extrae las columnas específicas en un DataFrame
    """
    try:
        print("🚀 Conectando a MySQL...")
        
        # Establecer conexión
        conexion = pymysql.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME,
            charset='utf8mb4'
        )
        
        print(f"✅ Conectado a MySQL y usando la base: {DB_NAME}")
        
        # Query para extraer solo las columnas que necesitas
        query = """
            WITH RECURSIVE subordinados AS (
                SELECT
                    jefes.rut AS jefe_rut,
                    empleados.rut AS empleado_rut,
                    1 AS nivel
                FROM rrhh_app.employees AS empleados
                JOIN rrhh_app.employees AS jefes ON empleados.rut_boss = jefes.rut
                WHERE empleados.status = 'activo' AND jefes.status = 'activo'

                UNION ALL

                SELECT
                    s.jefe_rut,
                    e.rut,
                    s.nivel + 1
                FROM subordinados s
                JOIN rrhh_app.employees e ON e.rut_boss = s.empleado_rut
                WHERE e.status = 'activo' AND s.nivel < 5
            ),

            jefes_info AS (
                SELECT DISTINCT
                    jefes.rut,
                    jefes.full_name,
                    jefes.name_role,
                    jefes.rut_boss,
                    jefes.cost_center,
                    jefes.area_id,
                    jefes.base_wage,
                    jefe_superior.full_name AS reporta_a,
                    jefe_superior.name_role AS cargo_jefe,
                    areas.first_level_name AS empresa,
                    areas.second_level_name AS seccion,
                    areas.name AS area
                FROM rrhh_app.employees AS jefes
                LEFT JOIN rrhh_app.employees AS jefe_superior ON jefes.rut_boss = jefe_superior.rut
                LEFT JOIN rrhh_app.areas AS areas ON jefes.cost_center = areas.cost_center AND jefes.area_id = areas.id
                WHERE jefes.status = 'activo'
                AND (
                    jefes.rut IN (SELECT DISTINCT jefe_rut FROM subordinados)
                    OR jefes.name_role = 'Supervisor de Planta'
                )
            ),

            percentiles AS (
                SELECT
                    rut,
                    base_wage,
                    PERCENT_RANK() OVER (ORDER BY base_wage) AS percentil
                FROM rrhh_app.employees
                WHERE status = 'activo'
            )

            SELECT
                ji.full_name AS nombre,
                ji.name_role AS cargo,
                COUNT(DISTINCT CASE WHEN subordinados.nivel = 1 THEN subordinados.empleado_rut END) AS personas_directas,
                COUNT(DISTINCT CASE WHEN subordinados.nivel > 1 THEN subordinados.empleado_rut END) AS personas_indirectas,
                ji.reporta_a,
                ji.cargo_jefe,
                ji.empresa,
                ji.seccion,
                ji.area,
                ROUND(p.percentil * 100, 2) AS percentil_sueldo_base
            FROM jefes_info ji
            LEFT JOIN subordinados ON ji.rut = subordinados.jefe_rut
            LEFT JOIN percentiles p ON ji.rut = p.rut
            GROUP BY ji.full_name, ji.name_role, ji.rut_boss,
                    ji.reporta_a, ji.cargo_jefe,
                    ji.empresa, ji.seccion, ji.area,
                    p.percentil
            ORDER BY personas_directas DESC, personas_indirectas DESC;
            """
        
        # Crear DataFrame directamente desde la query
        df = pd.read_sql(query, conexion)
        
        print(f"📋 Columnas disponibles: {list(df.columns)}")
        
        # Cerrar conexión
        conexion.close()
        print("✅ Conexión cerrada correctamente.")
        
        return df
        
    except Exception as e:
        print(f"❌ Error al conectar o extraer datos: {e}")
        return None

In [3]:

def export_query_to_excel(df):
    # Exportar DataFrame a Excel
    df.to_excel('resultado_query_lizeth.xlsx', index=False)
    print("Archivo Excel generado: resultado_query_lizeth.xlsx")

df = conectar_y_extraer_datos()
if df is not None:
    export_query_to_excel(df)

🚀 Conectando a MySQL...
✅ Conectado a MySQL y usando la base: rrhh_app


C:\Users\gpavez\AppData\Local\Temp\ipykernel_21592\926701063.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conexion)


📋 Columnas disponibles: ['cargo', 'nombre', 'correo']
✅ Conexión cerrada correctamente.
Archivo Excel generado: resultado_query_lizeth.xlsx
